In [ ]:
from clearml import Task, PipelineController
from dataclasses import dataclass
# force colab to get dataclasses incase you are running worker in colab
Task.add_requirements('dataclasses')

In [ ]:
base_project_name = 'titanic_demo'

@dataclass
class PipeConfig:
    featurestore_subproject: str = "FeatureStore"
    featureset_creation: str = "feature_set_2"

In [ ]:
task = Task.init(
    project_name=base_project_name,
    task_name='Pipeline automation',
    output_uri=True,  # auto save everything to Clearml Free
)

pipe_cfg = PipeConfig()
task.connect(pipe_cfg, 'pipeline config')

In [ ]:
pipe = PipelineController(default_execution_queue='default')
# step 0 - new data in dataset subproject
# not included (see video for explanations)
# step 1 - create feature set using your own code
pipe.add_step(name='feature_set_creation',
              base_task_project=f'{base_project_name}/{pipe_cfg.featurestore_subproject}',
              base_task_name=f'{pipe_cfg.featureset_creation}'
              )
# step 2 - use feature set with your own code
pipe.add_step(name='use_feature_Set',
              base_task_project=f'{base_project_name}',
              base_task_name='S02E07: feature_set for automation',
              parents=['feature_set_creation', ],
              )

# Starting the pipeline (in the background)
print('starting...')
pipe.start()
# Wait until pipeline terminates
pipe.wait()
# cleanup everything
pipe.stop()

print('done')